In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re
import rule_utils as ru

In [2]:
df = pd.read_csv('../data/input.csv')
df.head(2)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,9,9,9,9,9,0,9,9,9,9
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,9,9,0,9,9,9,9,9,9,9


价格 ：in: 1161 not in: 112 out: 177, 1273.
内饰：in: 484 not in: 52 out: 109, 536
动力：in: 2405 not in: 327 out: 651, 2732.0
外观in: 449 not in: 40 out: 135, 489.0
安全性	in: 533 not in: 40 out: 302, 573.
操控in: 707 not in: 329 out: 206, 1036.0	
油耗in: 1022 not in: 60 out: 82, 1082.0
空间in: 427 not in: 15 out: 124, 442.0
舒适性in: 814 not in: 117 out: 497, 931.0
配置in: 770 not in: 83 out: 342, 853.0	

In [125]:
actuall_price = df[df['配置']!=9]
actual_has = set(actuall_price.index)

In [168]:
isprice = ru.match_subject('../rules/config.txt')

In [169]:
has_price = df['content'].apply(lambda v: 1 if isprice(v) else 0)

In [170]:
pred_has = set(has_price[has_price!=0].index)

In [171]:
print(len(actual_has), len(pred_has))

853 1112


In [172]:
print('in:', len(actual_has&pred_has), 'not in:', len(actual_has-pred_has), 'out:', len(pred_has-actual_has))

in: 770 not in: 83 out: 342


In [173]:
# FP = df.reindex(list(pred_has-actual_has))
# for v in FP['content']:
#     print(v)

In [174]:
# TN = df.reindex(list(actual_has-pred_has))
# for v in TN['content']:
#     print(v)

### 测试

In [175]:
subjects = {
    'price': '价格', 
    'interior': '内饰', 
    'power': '动力', 
    'surface': '外观', 
    'safety': '安全性', 
    'operation':'操控', 
    'gas' : '油耗', 
    'space': '空间', 
    'comfort': '舒适性', 
    'config': '配置'
}


In [179]:
matchers = [(sub, ru.match_subject('../rules/{}.txt'.format(sub))) for sub in subjects]

In [177]:
test = pd.read_csv('../data/test_public.csv')
test.head(5)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [194]:
with open('../output/rule1.csv', 'w') as f:
    first = True
    line = '{},{},0,'
    f.write('content_id,subject,sentiment_value,sentiment_word')
    for index, row in test.iterrows():
        has = False
        for sub, matcher in matchers:
            
            if matcher(row['content']):
                has = True
                value = line.format(row['content_id'], subjects[sub])
                f.write('\n')
                f.write(value)
        if not has:
            value = line.format(row['content_id'], '价格')
            f.write('\n')
            f.write(value)


In [191]:
df = pd.read_csv('../output/rule1.csv', index_col='content_id')
output_index = set(df.index)
input_index = set(test['content_id'])

In [192]:
input_index - output_index

set()

In [193]:
with open('../output/baseline.csv', 'w') as f:
    first = True
    line = '{},{},0,'
    f.write('content_id,subject,sentiment_value,sentiment_word')
    for index, row in test.iterrows():
        value = line.format(row['content_id'], '价格')
        f.write('\n')
        f.write(value)

In [ ]:
with open('../output/rule1v2.csv', 'w') as f:
    first = True
    line = '{},{},0,'
    f.write('content_id,subject,sentiment_value,sentiment_word')
    for index, row in test.iterrows():
        has = False
        for sub, matcher in matchers:
            
            if matcher(row['content']):
                has = True
                value = line.format(row['content_id'], subjects[sub])
                f.write('\n')
                f.write(value)
        if not has:
            for sub in subjects.values():
                value = line.format(row['content_id'], sub)
                f.write('\n')
                f.write(value)
